# Import of packages

In [1]:
'''To install the required packages that are not included in Colaboratory.'''
#Installation of packages that are not included in Colaboratory
!pip install kneed
!pip install osmnx

'''To install a specific version of the matplotlib package.'''
!pip install matplotlib==3.1.3

'''To import the required packages.'''
import pandas as pd
import numpy as np
import networkx as nx
import collections
import matplotlib.pyplot as plt
import math
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import os
import matplotlib.image as mpimg
import statistics

from kneed import KneeLocator
import seaborn as sns
from sklearn.cluster import KMeans
from scipy.spatial import distance
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA

import osmnx as ox

  Using cached matplotlib-3.4.1-cp38-cp38-macosx_10_9_x86_64.whl (7.2 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.1.3
    Uninstalling matplotlib-3.1.3:
      Successfully uninstalled matplotlib-3.1.3
  Using cached matplotlib-3.1.3-cp38-cp38-macosx_10_9_x86_64.whl (13.2 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.4.1
    Uninstalling matplotlib-3.4.1:
      Successfully uninstalled matplotlib-3.4.1
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

osmnx 1.0.1 requires matplotlib>=3.3, but you'll have matplotlib 3.1.3 which is incompatible.


/Users/pol/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.neighbors.nearest_centroid module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


# Settings

In [2]:
'''To display all output results of a Jupyter cell.'''
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
'''To ensure that the output results of extensive output results are not truncated.'''
#pd.options.display.max_rows = 4000

'To ensure that the output results of extensive output results are not truncated.'

# File locations


In [4]:
'''If your computer is a Windows and that you are using the file locally (repository_loc == os.getcwd()) put True, False otherwise'''
windows_locally = False 

'If your computer is a Windows and that you are using the file locally (repository_loc == os.getcwd()) put True, False otherwise'

In [5]:
'''Register the GitHub link or the file relative location (select one)'''
#the Github link
#repository_loc, windows_locally = "https://raw.githubusercontent.com/polkuleuven/Thesis_Train/main", False
#the local link
repository_loc = os.getcwd()

'Register the GitHub link or the file relative location (select one)'

In [6]:
'''Get the other folder locations'''

stops_cleaned_loc = repository_loc + '/stops_cleaned/'
df_for_edges_loc = repository_loc + '/df_for_edges/'
routes_loc = repository_loc + '/routes/'
coutries_train_networks_loc = repository_loc + '/countries_train_networks'
images_loc = repository_loc + '/images/'

if windows_locally:
    stops_cleaned_loc = stops_cleaned_loc.replace('/', " \\ ").replace(' ', "")
    df_for_edges_loc = df_for_edges_loc.replace('/', " \\ ").replace(' ', "")
    routes_loc = routes_loc.replace('/', " \\ ").replace(' ', "")    
    coutries_train_networks_loc = coutries_train_networks_loc.replace('/', "\\ ").replace(' ', "") 
    images_loc = images_loc.replace('/', "\\ ").replace(' ', "") 

'Get the other folder locations'

# Import of the DataFrames

In [7]:
''' To import the df_for_edges dfs'''
df_for_edges_Belgium = pd.read_csv( df_for_edges_loc + 'df_for_edges_Belgium.csv', sep=",").drop(columns = ['Unnamed: 0'])
df_for_edges_Netherlands = pd.read_csv(df_for_edges_loc + 'df_for_edges_Netherlands.csv', sep=",").drop(columns = ['Unnamed: 0'])
df_for_edges_Switzerland = pd.read_csv(df_for_edges_loc + 'df_for_edges_Switzerland.csv', sep=",").drop(columns = ['Unnamed: 0'])

' To import the df_for_edges dfs'

In [8]:
''' To import the stops_cleaned df'''
stops_cleaned_Belgium = pd.read_csv(stops_cleaned_loc + 'stops_cleaned_Belgium.csv', sep=",").drop(columns = ['Unnamed: 0'])
stops_cleaned_Netherlands = pd.read_csv(stops_cleaned_loc + 'stops_cleaned_Netherlands.csv', sep=",")
stops_cleaned_Switzerland = pd.read_csv(stops_cleaned_loc + 'stops_cleaned_Switzerland.csv', sep=",")

' To import the stops_cleaned df'

In [9]:
''' To import routes df'''
belgian_routes_Belgium = pd.read_csv(routes_loc + 'belgian_routes_Belgium.csv', sep=",").drop(columns = ['Unnamed: 0'])
dutch_routes_Netherlands = pd.read_csv(routes_loc + 'dutch_routes_Netherlands.csv', sep=",").drop(columns = ['Unnamed: 0'])
swiss_routes_Switzerland = pd.read_csv(routes_loc + 'swiss_routes_Switzerland.csv', sep=",").drop(columns = ['Unnamed: 0'])

' To import routes df'

# Research question 1: How similar are railway systems in terms of their global network metrics? Comparison of the railway systems in space-of-stops representation

## Functions

In [10]:
''' To construct the network graph for the country '''
def create_graph(df_for_edges):
    G = nx.Graph()
    G = nx.from_pandas_edgelist(df_for_edges, 'stop_name_1', 'stop_name_2')
    return G

' To construct the network graph for the country '

In [11]:
''' To assign weights to the network graph for the country'''
def assign_weights (G, df_for_edges, stops_cleaned):
    # To assign node weights to graph
    for i in range(len(stops_cleaned)):
        for n, d in G.nodes(data=True):
            if stops_cleaned.stop_name[i] == n:
                G.nodes[n]['id']=stops_cleaned.stop_id[i]
                G.nodes[n]['latitude']=stops_cleaned.stop_lat[i]
                G.nodes[n]['longitude']=stops_cleaned.stop_lon[i]
                G.nodes[n]['country']=stops_cleaned.country[i]

    # To assign edge weights to the graph: the edge weight represents a list that contains all route_ids that cover that edge
    routes_on_edge = {}
    for node1,node2, edgeData in G.edges(data = True):
        if (node1, node2) not in routes_on_edge:
                routes_on_edge[(node1, node2)] = set()
        route_id_node1_node2= df_for_edges[(df_for_edges['stop_name_1'] == node1 ) & (df_for_edges['stop_name_2'] == node2 )]['route_id']
        route_id_node2_node1= df_for_edges[(df_for_edges['stop_name_1'] == node2 ) & (df_for_edges['stop_name_2'] == node1 )]['route_id']
        for route_id in pd.concat([route_id_node1_node2, route_id_node2_node1]):
            routes_on_edge[(node1, node2)].add(route_id)
        edgeData['route_id'] = routes_on_edge[(node1, node2)] 

    return G

' To assign weights to the network graph for the country'

In [12]:
'''Calculates the average degree'''
def avg_degree(G):
    sum_degrees = 0
    for n in G.nodes():
        sum_degrees += G.degree[n]
    return (sum_degrees/len(G.nodes()))

'Calculates the average degree'

In [13]:
'''Calculates the gamma index'''
def gamma_index(G):
    return len(G.edges()) / (3*(len(G.nodes()) - 2))

'Calculates the gamma index'

In [14]:
''' The degree distribution of the country railway system'''
def degree_distribution(G):

    #Sort degree values
    degree_sequence=sorted([d for n, d in G.degree()], reverse=True)

    #Count number of frequences of degree values
    degreeCount=collections.Counter(degree_sequence)

    #Define variables deg and cnt and probability
    deg, cnt = zip(*degreeCount.items())
    probability = []
    for v in cnt:
        pv= v/len(G.nodes())
        probability.append(pv)

    #plot degree probability histogram
    fig, ax = plt.subplots(figsize=(5, 5))
    plt.bar(deg, probability, width=0.95, color='lightgrey')

    #plot degree distribution
    plt.plot(deg, probability, color="b")

    #graph settings
    plt.title("Degree distribution")
    plt.ylabel("P (k)")
    plt.xlabel("k")
    ax.set_xticks([d for d in deg])
    ax.set_xticklabels(deg)

    #count
    fig, ax = plt.subplots(figsize=(5, 5))
    plt.bar(deg, cnt, width=0.95, color='lightgrey')

    plt.plot(deg, cnt, color="b")

    #graph settings
    plt.title("Degree count")
    plt.ylabel("Count")
    plt.xlabel("k")
    ax.set_xticks([d for d in deg])
    ax.set_xticklabels(deg)

    plt.show()

' The degree distribution of the country railway system'

In [15]:
'''Creates the railway network (using the self-made Networkx graph) of the country on the country map in red. 
The white on that map is the actual railway network given by OpenStreetmaps'''

import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox

def plot_given_Networkx_on_country_map(country_Networkx, country_name, country_names, dic_country_networks, dic_country_names):
    
    train_network = dic_country_networks[country_names[0]]
    country_shape = ox.geocode_to_gdf(dic_country_names[country_names[0]])
    for country in country_names[1:]:
        country_shape = pd.concat([country_shape, ox.geocode_to_gdf(dic_country_names[country])])
        train_network = nx.compose(train_network, dic_country_networks[country])

    #create an empty list of edges
    list_edges = []
    
    #add each edge to the list, by finding both of its nodes in the given OpenStreetMaps network and highlighting them
    #and finding the shortest routes between each pair of edges to highlight that route as well
    for node1,node2 in country_Networkx.edges():
        orig = ox.get_nearest_node(train_network, (country_Networkx.nodes[node1]['latitude'], country_Networkx.nodes[node1]['longitude']))
        dest = ox.get_nearest_node(train_network, (country_Networkx.nodes[node2]['latitude'], country_Networkx.nodes[node2]['longitude']))
        try:
            list_edges.append(nx.shortest_path(train_network, orig, dest))
        except nx.NetworkXNoPath:
            print(f'No path between {node1} and {node2}')
            continue

    #put the self-made railway network as a graph in red
    fig, ax = ox.plot_graph_routes(train_network, list_edges, route_color = 'r', route_linewidth = 1, show=False, close=False, edge_color='#111111', node_color='#111111', node_size=0.2, bgcolor='w')

    #plot the country on the top of the self-made railway network
    country_shape.plot(ax=ax, color='#111111', alpha=0.4)

    plt.savefig(f'{images_loc}{country_name}_Network.png') 
    plt.show()

'Creates the railway network (using the self-made Networkx graph) of the country on the country map in red. \nThe white on that map is the actual railway network given by OpenStreetmaps'

## Download the train networks of all the counties of interest

### Functions

In [16]:
'''Download all the countries train networks and store it as .gpickles files'''

def download_countries_train_Networks(dic_country_names, coutries_train_networks_loc):
    for country, osm_request in dic_country_names.items():
        train_country = ox.graph_from_place(osm_request,
                                            retain_all=False, truncate_by_edge=True, simplify=True,
                                            custom_filter='["railway"~"rail"]')
        nx.write_gpickle(train_country, f'{coutries_train_networks_loc}/train_{country}.gpickle')
    

'Download all the countries train networks and store it as .gpickles files'

In [17]:
'''Retrieve all the countries train networks and put it in the dic_country_networks dictionnary'''

def retrieve_countries_train_Networks(dic_country_names, coutries_train_networks_loc):
    dic_country_networks = {}
    for country in dic_country_names:
        train_country = nx.read_gpickle(f'{coutries_train_networks_loc}/train_{country}.gpickle')
        dic_country_networks[country] = train_country
    return dic_country_networks
    

'Retrieve all the countries train networks and put it in the dic_country_networks dictionnary'

### Acutal download

In [18]:
'''Create a dictionary with all the countries of interest'''
dic_country_names = {'Netherlands' : ['South Holland, Netherlands',{'state':'Utrecht', 'country':'Netherlands'},'North Brabant, Netherlands', 'Limburg, Netherlands','Gelderland, Netherlands', 'Overijssel, Netherlands','Drenthe, Netherlands', 'Groningen, Netherlands', 'Friesland, Netherlands', 'Flevoland, Netherlands', 'North Holland, Netherlands'],
                    'Belgium': ['Belgium'], 'Switzerland': ['Switzerland'], 'France': ['Metropolitan France'],
                    'Luxembourg': ['Luxembourg'], 'Germany': ['Germany'], 'Austria': ['Austria'], 'Italy': ['Italy'],
                    'Liechtenstein': ['Liechtenstein'], 'United_Kingdom': ['United Kingdom']}

'Create a dictionary with all the countries of interest'

In [19]:
#THE FOLLOWING LINE MIGHT BE RUNNED IF WANTED TO DOWNLOAD THE COUNTRIES TRAIN NETWORKS FROM OSM, BUT IT TAKES A LONG TIME
#download_countries_train_Networks(dic_country_names, coutries_train_networks_loc)

In [20]:
dic_country_networks = retrieve_countries_train_Networks(dic_country_names, coutries_train_networks_loc)

## Acutal calculation of the global characteristics of the railway systems in the space-of-stops representation

### Belgium

In [21]:
B = create_graph(df_for_edges_Belgium)
B = assign_weights (B, df_for_edges_Belgium, stops_cleaned_Belgium)

In [22]:
'''Calculates the subgraph that contains only stops into Belgium iteself (removes all the stop outside the country)'''
belgian_stops = [n for n,v in B.nodes(data=True) if v['country'] == 'Belgium']
B_belgian_stops = B.subgraph(belgian_stops)

'Calculates the subgraph that contains only stops into Belgium iteself (removes all the stop outside the country)'

In [ ]:
'''Show the country train network that was created by create_graph() on the map'''
country_name = 'Belgium'
country_names = ['France', 'Belgium', 'Netherlands', 'Luxembourg', 'Germany']
#THE FOLLOWING LINE MIGHT BE RUNNED IF WANTED TO RECALCULATE AND SHOW THE ROUTE CREATED ON THE MAP, BUT IT TAKES A LONG TIME
plot_given_Networkx_on_country_map(B, country_name, country_names, dic_country_networks, dic_country_names)
#img = mpimg.imread(f'{images_loc}{country_name}_Network.png')
#imgplot = plt.imshow(img)
#plt.show()

'Show the country train network that was created by create_graph() on the map'

In [ ]:
'''Show the country train network subgraph that was created by create_graph() and then filtered on the country stops on the map'''
country_name = 'Belgium_only'
country_names = ['Belgium']
#THE FOLLOWING LINE MIGHT BE RUNNED IF WANTED TO RECALCULATE AND SHOW THE ROUTE CREATED ON THE MAP, BUT IT TAKES A LONG TIME
plot_given_Networkx_on_country_map(B_belgian_stops, country_name, country_names, dic_country_networks, dic_country_names)
#img = mpimg.imread(f'{images_loc}{country_name}_Network.png')
#imgplot = plt.imshow(img)
#plt.show()

In [ ]:
B.edges(data=True)

In [ ]:
''' The number of nodes in the Belgian railway system '''
# The number of total nodes in the Belgian railway system that consists of all routes that contain at least one node in Belgium
len(B.nodes())

# The number of belgian nodes in the Belgian railway system that consists of all routes that contain at least one node in Belgium
len(B_belgian_stops.nodes())

In [ ]:
''' The degrees of the nodes in the Belgian railway system considering only stops in Belgium'''
B_belgian_stops.degree()

In [ ]:
''' The average degree of the nodes in the Belgian railway system indicating the number of directions 
a commuter can take from that given station.'''
# The average degree of all the nodes in the Belgian railway system that consists of all routes that cover at least one belgian station
print(avg_degree(B))

# The average degree of all the nodes in the Belgian railway system if only the belgian stations are considered
print(avg_degree(B_belgian_stops))

In [ ]:
''' The number of edges in the Belgian railway system '''
len(B.edges())

In [ ]:
''' The gamma index of the Belgian railway system '''
print(gamma_index(B))

In [ ]:
''' The mean clustering coefficient of the Belgian railway system '''
#The mean clustering coefficient indicates for a random station the estimation of the division of the number of neighbouring stations of a station that are connected 
# divided by the possible number of connections between the neighbours of that station
print ("mean clustering coefficient:", nx.average_clustering(B))

In [ ]:
''' The degree distribution of the Belgian railway system considering only stops in Belgian'''
degree_distribution(B_belgian_stops)

In [ ]:
''' Detection of the scale-free property in the Belgian railway system:
A PTN with the scale-free property consists of a large number of vertices 
that exhibit low degree and a few number of vertices that exhibit very high degree. '''
print("The scale-free property is present in the Belgian railway network because the majority of nodes exhibit a low degree value equal to 2 and almost none of the nodes exhibit a degree value equal to 5 or higher")

In [ ]:
''' The average path length of the Belgian railway system :
The average path length represents in the space_of_stops the average of all the shortest paths 
to be transversed between all public transport stop pairs
which can be reached by means of one or multiple routes.'''
print(nx.average_shortest_path_length(B))

In [ ]:
''' The assortativity of the Belgian railway system considering only stops in Belgium:
In an assortative network, high-degree vertices tend to be connected and low-degree vertices tend to stick together as well. 
In a disassortative network conversely, high-degree and low-degree vertices tend to be connected 
to each other instead of being connected to vertices of the same kind.'''
print(nx.degree_pearson_correlation_coefficient(B_belgian_stops))

### Netherlands

In [ ]:
N = create_graph(df_for_edges_Netherlands)
N = assign_weights(N, df_for_edges_Netherlands, stops_cleaned_Netherlands)

In [ ]:
'''Calculates the subgraph that contains only stops into Netherlands iteself (removes all the stop outside the country)'''
dutch_stops = [n for n,v in N.nodes(data=True) if v['country'] == 'Netherlands']
N_dutch_stops = N.subgraph(dutch_stops)

In [ ]:
'''Show the country train network that was created by create_graph() on the map'''
country_name = 'Netherlands'
country_names = ['France', 'Belgium', 'Netherlands', 'United_Kingdom', 'Germany', 'Austria', 'Switzerland']
#THE FOLLOWING LINE MIGHT BE RUNNED IF WANTED TO RECALCULATE AND SHOW THE ROUTE CREATED ON THE MAP, BUT IT TAKES A LONG TIME
plot_given_Networkx_on_country_map(N, country_name, country_names, dic_country_networks, dic_country_names)
#img = mpimg.imread(f'{images_loc}{country_name}_Network.png')
#imgplot = plt.imshow(img)
#plt.show()

In [ ]:
'''Show the country train network subgraph that was created by create_graph() and then filtered on the country stops on the map'''
country_name = 'Netherlands_only'
country_names = ['Netherlands']
#THE FOLLOWING LINE MIGHT BE RUNNED IF WANTED TO RECALCULATE AND SHOW THE ROUTE CREATED ON THE MAP, BUT IT TAKES A LONG TIME
plot_given_Networkx_on_country_map(N_dutch_stops, country_name, country_names, dic_country_networks, dic_country_names)
#img = mpimg.imread(f'{images_loc}{country_name}_Network.png')
#imgplot = plt.imshow(img)
#plt.show()

In [ ]:
N.edges(data=True)

In [ ]:
''' The number of nodes in the Dutch railway system '''
# The number of total nodes in the Dutch railway system that consists of all routes that contain at least one node in the Netherlands
len(N.nodes())

# The number of dutch nodes in the Dutch railway system that consists of all routes that contain at least one node in the Netherlands
print(len(N_dutch_stops.nodes()))

In [ ]:
''' The degrees of the nodes in the Dutch railway system considering only stops in Netherlands'''
N_dutch_stops.degree()

In [ ]:
''' The average degree of the nodes in the Dutch railway system indicating the number of directions 
a commuter can take from that given station.'''
# The average degree of all the nodes in the Dutch railway system that consists of all routes that cover at least one dutch station
print(avg_degree(N))

# The average degree of all the nodes in the Dutch railway system if only the dutch stations are considered
print(avg_degree(N_dutch_stops))

In [ ]:
''' The number of edges in the Dutch railway system '''
len(N.edges())

In [ ]:
''' The gamma index of the Dutch railway system '''
print(gamma_index(N))

In [ ]:
''' The mean clustering coefficient of the Dutch railway system '''
#The mean clustering coefficient indicates for a random station the estimation of the division of the number of neighbouring stations of a station that are connected 
# divided by the possible number of connections between the neighbours of that station
print ("mean clustering coefficient:", nx.average_clustering(N))

In [ ]:
''' The degree distribution of the Dutch railway system considering only stops in Netherlands'''
degree_distribution(N_dutch_stops)

In [ ]:
''' Detection of the scale-free property in the Dutch railway system:
A PTN with the scale-free property consists of a large number of vertices that exhibit low degree and a few number of vertices that exhibit very high degree. '''
print("The scale-free property is present in the Dutch railway network because the majority of nodes exhibit a low degree value equal to 2 and almost none of the nodes exhibit a degree value equal to 6 or higher")

In [ ]:
''' The average path length of the Dutch railway system :
The average path length represents in the space-of-stops the average of all the shortest paths 
to be transversed between all public transport stop pairs
which can be reached by means of one or multiple routes.'''
print(nx.average_shortest_path_length(N))

In [ ]:
''' The assortativity of the Dutch railway system considering only the stops in Netherlands:
In an assortative network, high-degree vertices tend to be connected and low-degree vertices tend to stick together as well. 
In a disassortative network conversely, high-degree and low-degree vertices tend to be connected 
to each other instead of being connected to vertices of the same kind.'''
print(nx.degree_pearson_correlation_coefficient(N_dutch_stops))

### Switzerland

In [ ]:
S = create_graph(df_for_edges_Switzerland)
S = assign_weights(S, df_for_edges_Switzerland, stops_cleaned_Switzerland)

In [ ]:
'''Calculates the subgraph that contains only stops into Switzerland iteself (removes all the stop outside the country)'''
swiss_stops = [n for n,v in S.nodes(data=True) if v['country'] == 'Switzerland']
S_swiss_stops = S.subgraph(swiss_stops)

In [ ]:
'''Show the country train network that was created by create_graph() on the map'''
country_name = 'Switzerland'
country_names = ['France', 'Germany', 'Austria', 'Switzerland', 'Italy', 'Liechtenstein']
#THE FOLLOWING LINE MIGHT BE RUNNED IF WANTED TO RECALCULATE AND SHOW THE ROUTE CREATED ON THE MAP, BUT IT TAKES A LONG TIME
plot_given_Networkx_on_country_map(S, country_name, country_names, dic_country_networks, dic_country_names)
#img = mpimg.imread(f'{images_loc}{country_name}_Network.png')
#imgplot = plt.imshow(img)
#plt.show()

In [ ]:
'''Show the country train network subgraph that was created by create_graph() and then filtered on the country stops on the map'''
country_name = 'Switzerland_only'
country_names = ['Switzerland']
#THE FOLLOWING LINE MIGHT BE RUNNED IF WANTED TO RECALCULATE AND SHOW THE ROUTE CREATED ON THE MAP, BUT IT TAKES A LONG TIME
plot_given_Networkx_on_country_map(S_swiss_stops, country_name, country_names, dic_country_networks, dic_country_names)
#img = mpimg.imread(f'{images_loc}{country_name}_Network.png')
#imgplot = plt.imshow(img)
#plt.show()

In [ ]:
S.edges(data=True)

In [ ]:
''' The number of nodes in the Swiss railway system '''
# The number of total nodes in the Swiss railway system that consists of all routes that contain at least one node in the Netherlands
len(S.nodes())

# The number of swiss nodes in the Swiss railway system that consists of all routes that contain at least one node in the Netherlands
print(len(S_swiss_stops.nodes()))

In [ ]:
''' The degrees of the nodes in the Swiss railway system considering only stops in Switzerland'''
S_swiss_stops.degree()

In [ ]:
''' The average degree of the nodes in the Swiss railway system indicating the number of directions 
a commuter can take from that given station.'''
# The average degree of all the nodes in the Swiss railway system that consists of all routes that cover at least one Swiss station
print(avg_degree(S))

# The average degree of all the nodes in the Swiss railway system if only the Swiss stations are considered
print(avg_degree(S_swiss_stops))

In [ ]:
''' The number of edges in the Swiss railway system '''
len(S.edges())

In [ ]:
''' The gamma index of the Swiss railway system '''
print(gamma_index(S))

In [ ]:
''' The mean clustering coefficient of the Swiss railway system '''
#The mean clustering coefficient indicates for a random station the estimation of the division of the number of neighbouring stations of a station that are connected 
# divided by the possible number of connections between the neighbours of that station
print ("mean clustering coefficient:", nx.average_clustering(S))

In [ ]:
''' The degree distribution of the Swiss railway system considering only stops in Switzerland'''
degree_distribution(S_swiss_stops)

In [ ]:
''' Detection of the scale-free property in the Swiss railway system:
A PTN with the scale-free property consists of a large number of vertices 
that exhibit low degree and a few number of vertices that exhibit very high degree. '''
print("The scale-free property is present in the Swiss railway network because the majority of nodes exhibit a low degree value equal to 2 and almost none of the nodes exhibit a degree value equal to 5 or higher")

In [ ]:
'''Separate all the connected subgraphs of Switzerland. And see what we lose by only taking the largest one.'''
all_connected_graphs = nx.connected_components(S)
largest_component = max(nx.connected_components(S), key=len)
largest_connected_subgraph = S.subgraph(largest_component)
lost_number_nodes = len(S.nodes()) - len(largest_connected_subgraph)
print(f'lost number of nodes: {lost_number_nodes}')
list_subgraphs_except_max = []
for sg in all_connected_graphs:
    if sg != largest_component:
        list_subgraphs_except_max.append(sg)

set_deleted_routes = set()
for sg in list_subgraphs_except_max:
    for n1, n2, edge_data in S.subgraph(sg).edges(data=True):
        set_deleted_routes.update(edge_data['route_id'])
        
print(f'lost number of routes: {len(set_deleted_routes)}')

In [ ]:
''' The average path length of the Swiss railway system :
The average path length represents in the space-of-stops the average of all the shortest paths 
to be transversed between all public transport stop pairs
which can be reached by means of one or multiple routes.'''
print(nx.average_shortest_path_length(largest_connected_subgraph))

In [ ]:
''' The assortativity of the Swiss railway system considering only stops in Switzerland:
In an assortative network, high-degree vertices tend to be connected and low-degree vertices tend to stick together as well. 
In a disassortative network conversely, high-degree and low-degree vertices tend to be connected to each other instead of being connected to vertices of the same kind.'''
print(nx.degree_pearson_correlation_coefficient(S_swiss_stops))

# RQ2: comparison of the Belgian NMBS/SNCB, the Dutch NS and the Swiss SBB railway systems in terms of the average planned travel time of similar routes

## Clustering preparation

### Functions

In [ ]:
''' To put all the unique route_ids of the df_for_edges in df route_characteristics_Belgium'''
def create_route_characteristics(df_for_edges):
    # To create a list of all unique route_ids in the df_for_edges dataset
    routes_list = list(df_for_edges['route_id'].unique())
    routes_list.sort()

    # To create a dataframe that contains the values in the route_characteristics_Belgium_dic as the values in the first column'''
    route_characteristics = pd.DataFrame(routes_list, columns=['route_id'])
    return route_characteristics

In [ ]:
''' To put the stations per route_id in df route_characteristics_Belgium'''
def put_stations_route_characteristics(G, route_characteristics):
    # To construct a new column of the route_characteristics_Belgium dataset that has for every column value an empty set
    route_characteristics['nodes'] = [set() for _ in range(len(route_characteristics))]

    # To put the two nodes of an edge in the nodes column of the route_characteristics_Belgium dataset if the route_id of the corresponding row 
    # is present in the set of the list values of the corresponding edge that contains all route_ids that contain the edge
    for node1, node2 in G.edges():
        route_characteristics[['route_id','nodes']].apply(lambda x: x['nodes'].update((node1, node2)) if x['route_id'] in set(G.get_edge_data(node1, node2)['route_id']) else x, axis = 1)
    return route_characteristics

In [ ]:
'''Deletes every empty route (useful after a inside contry filter)'''
def deletes_empty_routes(route_characteristics):
    index_empty_routes = route_characteristics_Belgium_belgian_nodes[route_characteristics_Belgium_belgian_nodes['nodes'].map(lambda d: len(d)) == 0].index
    return route_characteristics.drop(index_empty_routes)

In [ ]:
''' Definitions to calculate route topological metrics'''

# The inputs of the get_metric_list function are a dictionary that contains for all node keys the corresponding metric values and a specific route_id of a column of 
# the route_characteristics dataset
# A new empty list metric_list is created for these inputs
# For the nodes that are in the route_characteristics dataset corresponding to this route_id, the corresponding metric values from the dictionary get appended to metric_list

from statistics import mean
def get_metric_list(route_characteristics, metric_dic, route_id):
    """this is a sub function that is used in calculate_metric_route_per_node"""
    metric_list = []
    for nodes in route_characteristics[route_characteristics['route_id'] == route_id]['nodes']:
        for node in nodes:
            metric_list.append(metric_dic[node])
    return metric_list

# For the metric, a new column is created in route_characteristics, where every column value consists of an empty list
# The values in the column of the df are determined by giving the metric_dic and the route_id of the row where that value is placed to the get_metric_list function
# The metric_list ultimately contains the metric values for a specific metric for all the stations corresponding to that route

def calculate_metric_route_per_node(route_characteristics, metric_dic, metric_name):
    """puts the metrics of each node of a route in a list in the route_characteristics df"""
    route_characteristics[metric_name] = [list for _ in range(len(route_characteristics))]
    route_characteristics[metric_name] = route_characteristics.apply(lambda x: get_metric_list(route_characteristics, metric_dic, x['route_id']) , axis = 1)
    
# The function starts from the metric_name, the original_metric_name and the aggregation_method inputs
# A new column named with the metric_name is created: initially it contains all NaN values
# Thereafter the aggregation_method is taken into account. The avg aggregation method returns for all column values the mean of the values in the corresponding row
# of the original_metric_name list. The max aggregation method returns for all column values the max of the values in the corresponding row of the original_metric_name list.

def calculate_metric_aggregation_route(route_characteristics, metric_name, original_metric_name, aggregation_method):
    '''calculate the metric aggregation of the routes, only avg and max are supported as string values for aggregation method
    metric_name is the aggregation column name and original_metric_name is the node metric column name'''
    route_characteristics[metric_name] = np.nan
    if aggregation_method == 'avg':
        route_characteristics[metric_name] = route_characteristics.apply(lambda x: mean(x[original_metric_name]) , axis = 1)
    elif aggregation_method == 'max':
        route_characteristics[metric_name] = route_characteristics.apply(lambda x: max(x[original_metric_name]) , axis = 1)

In [ ]:
'''Calculate the metrics of the route_characteristics df'''
def calculate_characteristics (G, route_characteristics):
    calculate_metric_route_per_node(route_characteristics, dict(G.degree()), 'node_degree')
    calculate_metric_aggregation_route(route_characteristics, 'avg_degree', 'node_degree', 'avg')
    calculate_metric_aggregation_route(route_characteristics, 'max_degree', 'node_degree', 'max')
    calculate_metric_route_per_node(route_characteristics, dict(nx.clustering(G)), 'node_clustering')
    calculate_metric_aggregation_route(route_characteristics, 'avg_clustering', 'node_clustering', 'avg')
    calculate_metric_aggregation_route(route_characteristics, 'max_clustering', 'node_clustering', 'max')
    calculate_metric_route_per_node(route_characteristics, dict(nx.closeness_centrality(G)), 'node_closeness_centrality')
    calculate_metric_aggregation_route(route_characteristics, 'avg_closeness_centrality', 'node_closeness_centrality', 'avg')
    calculate_metric_aggregation_route(route_characteristics, 'max_closeness_centrality', 'node_closeness_centrality', 'max')
    calculate_metric_route_per_node(route_characteristics, dict(nx.betweenness_centrality(G)), 'node_betweenness_centrality')
    calculate_metric_aggregation_route(route_characteristics, 'avg_betweenness_centrality', 'node_betweenness_centrality', 'avg')
    calculate_metric_aggregation_route(route_characteristics, 'max_betweenness_centrality', 'node_betweenness_centrality', 'max')
    return route_characteristics

In [ ]:
''' To add the number of stations present in the stop sequence of a route_id'''
def add_number_stations(route_characteristics):
    number_stations = []
    for number_stops in route_characteristics['nodes']:
        number_stations.append(len(number_stops))

    route_characteristics['len_stop_sequence'] = number_stations
    return route_characteristics

In [ ]:
''' To add the absolute distances between the starting and the ending station of all routes, and waiting and travel time'''

def add_absolute_distance_tt_wt(route_characteristics, routes_country):

    route_characteristics = route_characteristics.merge(routes_country[['route_id', 'distance','travel_time', 'waiting_time']], on='route_id')
    route_characteristics = route_characteristics.rename(columns = {'distance': 'absolute_distance'})
    return route_characteristics

In [ ]:
''' To calculate the relative distance of the stop sequence of the route by only taking the starting and the ending station into account'''

def add_relative_distance (route_characteristics):
    # The relative distance of a route is the distance of the route divided by the distance of the route with the longest distance
    relative_distance = []

    # To calculate the route which stop sequence has the greatest absolute distance
    max_distance = route_characteristics['absolute_distance'].max()

    # To calculate the relative distance of each route
    for abs_distance in route_characteristics['absolute_distance']:
        rel_distance = abs_distance / max_distance
        relative_distance.append(rel_distance)

    # To add the relative_distance list as a column to the route_characteristics dataset
    route_characteristics['relative_distance'] = relative_distance
    return route_characteristics

In [ ]:
''' To add the absolute distances between the starting and the ending station of all routes'''

# Remember that geographical locations that are located further away to the right of the Prime Meridian Greenwich instead of another location that is closer
# (but still to the right) of this Meridian Greenwich, have a greater longitute value.
# Remember that geographical locations that are higher above the Equator than other geographical locations (that are also above the Equator), have a 
# higher latitude value

def add_absolute_distance_begin_end(route_characteristics, df_for_edges, stops_cleaned):

    absolute_distance = []
    for route in route_characteristics['route_id']:
        subdf_stop_sequence = df_for_edges[df_for_edges['route_id'] == route]
        number_of_rows = len(subdf_stop_sequence.index)

        # To get the starting station and the ending_station of the stop sequence of a route
        starting_station = subdf_stop_sequence['stop_name_1'].iloc[0]
        ending_station = subdf_stop_sequence['stop_name_2'].iloc[number_of_rows - 1]

        # The radius of the earth
        R = 6373.0

        # To convert the coordinates in radians 
        lat_starting_station = math.radians(stops_cleaned[stops_cleaned['stop_name'] == starting_station]['stop_lat'].iloc[0])
        lon_starting_station = math.radians(stops_cleaned[stops_cleaned['stop_name'] == starting_station]['stop_lon'].iloc[0])
        lat_ending_station = math.radians(stops_cleaned[stops_cleaned['stop_name'] == ending_station]['stop_lat'].iloc[0])
        lon_ending_station = math.radians(stops_cleaned[stops_cleaned['stop_name'] == ending_station]['stop_lon'].iloc[0])

        # To calculate the change in coordinates
        dlon = lon_ending_station - lon_starting_station
        dlat = lat_ending_station - lat_starting_station

        # To use the Haversine formula to get the distance in kilometers between the starting_station and the ending_station
        a = math.sin(dlat / 2)**2 + math.cos(lat_starting_station) * math.cos(lat_ending_station) * math.sin(dlon / 2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

        # To calculate the distance
        distance = R * c

        # To add the distance in the absolute_distance list
        absolute_distance.append(distance)

    # To add the absolute_distance list as a column to the route_characteristics dataset
    route_characteristics['absolute_distance_begin_end'] = absolute_distance
    return route_characteristics

### Actual clustering preparation

#### Belgium

In [ ]:
'''Calculates the subgraph that contains only stops into Belgium iteself (removes all the stop outside the country)'''
belgian_stops = [n for n,v in B.nodes(data=True) if v['country'] == 'Belgium']
B_belgian_stops = B.subgraph(belgian_stops)

In [ ]:
route_characteristics_Belgium = create_route_characteristics(df_for_edges_Belgium)
route_characteristics_Belgium = put_stations_route_characteristics(B, route_characteristics_Belgium)
route_characteristics_Belgium = calculate_characteristics(B, route_characteristics_Belgium)
route_characteristics_Belgium = add_number_stations(route_characteristics_Belgium)
route_characteristics_Belgium = add_absolute_distance_tt_wt(route_characteristics_Belgium, belgian_routes_Belgium)
route_characteristics_Belgium = add_relative_distance (route_characteristics_Belgium)
route_characteristics_Belgium = add_absolute_distance_begin_end(route_characteristics_Belgium, df_for_edges_Belgium, stops_cleaned_Belgium)

In [ ]:
route_characteristics_Belgium

#### Netherlands

In [ ]:
'''Calculates the subgraph that contains only stops into Netherlands iteself (removes all the stop outside the country)'''
dutch_stops = [n for n,v in N.nodes(data=True) if v['country'] == 'Netherlands']
N_dutch_stops = N.subgraph(dutch_stops)

In [ ]:
route_characteristics_Netherlands = create_route_characteristics(df_for_edges_Netherlands)
route_characteristics_Netherlands = put_stations_route_characteristics(N, route_characteristics_Netherlands)
route_characteristics_Netherlands = calculate_characteristics(N, route_characteristics_Netherlands)
route_characteristics_Netherlands = add_number_stations(route_characteristics_Netherlands)
route_characteristics_Netherlands = add_absolute_distance_tt_wt(route_characteristics_Netherlands, dutch_routes_Netherlands)
route_characteristics_Netherlands = add_relative_distance (route_characteristics_Netherlands)
route_characteristics_Netherlands = add_absolute_distance_begin_end(route_characteristics_Netherlands, df_for_edges_Netherlands, stops_cleaned_Netherlands)

In [ ]:
route_characteristics_Netherlands

#### Switzerland

In [ ]:
'''Calculates the subgraph that contains only stops into Switzerland iteself (removes all the stop outside the country)'''
swiss_stops = [n for n,v in S.nodes(data=True) if v['country'] == 'Switzerland']
S_swiss_stops = S.subgraph(swiss_stops)

In [ ]:
route_characteristics_Switzerland = create_route_characteristics(df_for_edges_Switzerland)
route_characteristics_Switzerland = put_stations_route_characteristics(S, route_characteristics_Switzerland)
route_characteristics_Switzerland = calculate_characteristics(S, route_characteristics_Switzerland)
route_characteristics_Switzerland = add_number_stations(route_characteristics_Switzerland)
route_characteristics_Switzerland = add_absolute_distance_tt_wt(route_characteristics_Switzerland, swiss_routes_Switzerland)
route_characteristics_Switzerland = add_relative_distance (route_characteristics_Switzerland)
route_characteristics_Switzerland = add_absolute_distance_begin_end(route_characteristics_Switzerland, df_for_edges_Switzerland, stops_cleaned_Switzerland)

In [ ]:
route_characteristics_Switzerland

## Route clustering

The route_id is an identifying but not an explanatory variable and therefore not interesting.

The max_degree attribute is chosen because a route that contains a station from which there are many directions of travel may have a longer transfer time at this stop and thus a relatively higher travel_time of the route ceteris paribus.

The clustering attribute is not interesting in the context of route clustering. Stations with a high clustering value have immediate neighboring stations that share a link between them. This may indicate that a station is sometimes skipped on this trajectory. However, from the clustering coefficient, it is not possible to tell whether the route represents the route with many or few intermediate stops.

The closeness centrality is interesting because stations that have smaller shortests paths lengths to all  other stations are more connected and therefore often more important stations. These important stations have a higher transfer time.

The betweenness centrality is interesting because stations that lie more often on the paths between other stations are more connected and therefore often more important stations. These important stations have a higher transfer time.

The len_stop_sequence attribute is interesting because a higher number of stops results in a higher total transfer time ceteris paribus.

The absolute distance is interesting because trajectories that have a longer length will also typically take more time to travel.

The relative distance is not interesting as the scaled absolute distance is the same as the relative distance.

### Functions

In [ ]:
''' To select the interesting attributes for the clustering '''
def select_clustering_attributes(route_characteristics):
    clustering_df = route_characteristics[['max_degree', 'max_closeness_centrality', 'max_betweenness_centrality','len_stop_sequence', 'absolute_distance']]

    return clustering_df

In [ ]:
''' To scale all attributes such that their values are between 0 and 1 and save the result in a dataframe'''

def scale_clustering_attributes_to_df(clustering_df):
    scaled_clustering_df = clustering_df.copy()
    scaler = MinMaxScaler()

    # To scale the max_degree
    scaler.fit(scaled_clustering_df[['max_degree']])
    scaled_clustering_df['max_degree'] = scaler.transform(scaled_clustering_df[['max_degree']])

    # To scale the max_closeness_centrality
    scaler.fit(scaled_clustering_df[['max_closeness_centrality']])
    scaled_clustering_df['max_closeness_centrality'] = scaler.transform(scaled_clustering_df[['max_closeness_centrality']])

    # To scale the max_betweenness_centrality
    scaler.fit(scaled_clustering_df[['max_betweenness_centrality']])
    scaled_clustering_df['max_betweenness_centrality'] = scaler.transform(scaled_clustering_df[['max_betweenness_centrality']])

    # To scale the len_stop_sequence
    scaler.fit(scaled_clustering_df[['len_stop_sequence']])
    scaled_clustering_df['len_stop_sequence'] = scaler.transform(scaled_clustering_df[['len_stop_sequence']])

    # To scale the absolute distance
    scaler.fit(scaled_clustering_df[['absolute_distance']])
    scaled_clustering_df['absolute_distance'] = scaler.transform(scaled_clustering_df[['absolute_distance']])

    # To verify the ranges of the attributes
    scaled_clustering_df.describe()

    return scaled_clustering_df

In [ ]:
''' To scale all attributes such that their values are between 0 and 1 and save the result in an array'''

def scale_clustering_attributes_to_arr(clustering_df):

    scaler = MinMaxScaler()
    # To scale all attributes
    scaler.fit(clustering_df)
    scaled_clustering_arr = scaler.transform(clustering_df)

    return scaled_clustering_arr

In [ ]:
''' To create a reduced array with two components '''
def create_array_two_components(scaled_arr):
    # To create the array with the specified amount of desired components from the scaled_clustering_arr
    pca = PCA(n_components=2)
    pca.fit(scaled_arr)
    x_pca = pca.transform(scaled_arr)

    return x_pca

In [ ]:
''' To create a PCA plot'''
def create_PCA_plot_one_argument(x_pca):
    plt.figure(figsize=(8,6))
    plt.scatter(x_pca[:,0], x_pca[:,1])
    plt.xlabel("First principle component")
    plt.ylabel("Second principle component")

In [ ]:
''' To find the optimal number of clusters'''
def optimal_number_clusters(scaled_clustering_df):
    # The elbow method starts with a certain k and computes the sum of squared errors (sum of the squared distances of points from their cluster centroids).
    # Thereafter the elbow method computes this sum of squared errors also for other values of k.
    # The optimal k is indicated by the elow on the elbow plot.

    # To specify the k range
    k_range = range(1,50)

    # To introduce the array that will contain the sum of squared errors of all models
    sse = []

    # For each iteration, a new model is created with the number of clusters equal to k
    # The sum of squared error is given by the inertia parameter
    for k in k_range:
        km = KMeans(n_clusters=k)
        km.fit(scaled_clustering_df)
        sse.append(km.inertia_)

    # To plot the elbow plot
    plt.xlabel('k')
    plt.ylabel('Sum of squared error')
    plt.plot(k_range, sse)

    # To calculate the optimal number of clusters
    kl = KneeLocator(range(1, 50), sse, curve="convex", direction="decreasing")

    return kl.elbow

In [ ]:
''' To add the cluster group to the scaled_clustering_dfs and the clustering_dfs '''
def assign_cluster_group(optimal_number_clusters, clustering_df, scaled_clustering_df, x_pca_clustering_df):
    # To apply the clustering prediction
    km = KMeans(n_clusters=optimal_number_clusters)
    cluster_predicted = km.fit_predict(scaled_clustering_df)

    # To add the labels to the dfs
    clustering_cluster_groups_df = clustering_df.copy()
    clustering_cluster_groups_df['cluster_group'] = cluster_predicted

    scaled_clustering_cluster_groups_df = scaled_clustering_df.copy()
    scaled_clustering_cluster_groups_df['cluster_group'] = cluster_predicted

    x_pca_clustering_cluster_groups_df = x_pca_clustering_df.copy()
    x_pca_clustering_cluster_groups_df['cluster_group'] = cluster_predicted

    return km, clustering_cluster_groups_df, scaled_clustering_cluster_groups_df, x_pca_clustering_cluster_groups_df

In [ ]:
''' To get the cluster centroids '''
def find_cluster_centroids_in_5D(scaled_clustering_cluster_groups):
    # To retrieve the scaled attributes and to put them in an array
    X_df = scaled_clustering_cluster_groups.drop('cluster_group', axis=1)
    X = X_df.to_numpy()

    # To retrieve the labels and to put them in an array
    Y_df = scaled_clustering_cluster_groups['cluster_group']
    Y = Y_df.to_numpy()

    clf = NearestCentroid()
    clf.fit(X,Y)

    return clf.centroids_

In [ ]:
''' To get the cluster centroids '''
def find_cluster_centroids_in_2D(scaled_clustering_cluster_groups):
    # To retrieve the scaled attributes and to put them in an array
    X_df = scaled_clustering_cluster_groups[['Principal component 1', 'Principal component 2']]
    X = X_df.to_numpy()

    # To retrieve the labels and to put them in an array
    Y_df = scaled_clustering_cluster_groups['cluster_group']
    Y = Y_df.to_numpy()

    clf = NearestCentroid()
    clf.fit(X,Y)

    return clf.centroids_

In [ ]:
def get_cluster_centroids(km):
    return km.cluster_centers_

In [ ]:
def create_PCA_plot(x_pca, x_pca_centroids, scaled_clustering_cluster_groups_df):
    plt.figure(figsize=(8,6))
    plt.scatter(x_pca[:,0], x_pca[:,1], c=scaled_clustering_cluster_groups_df['cluster_group'])
    plt.scatter(x_pca_centroids[:,0], x_pca_centroids[:,1], color = 'red', marker='*', s=200)
    plt.xlabel("First principle component")
    plt.ylabel("Second principle component")

### Acutal clustering of 3 countries

In [ ]:
''' To create the clustering_dfs that contain the interesting attributes for clustering'''
clustering_df_Belgium = select_clustering_attributes(route_characteristics_Belgium)
clustering_df_Netherlands = select_clustering_attributes(route_characteristics_Netherlands)
clustering_df_Switzerland = select_clustering_attributes(route_characteristics_Switzerland)

''' To add a country attribute to the clustering_dfs'''
clustering_df_Belgium.loc[:,'country'] = 'Belgium'
clustering_df_Netherlands.loc[:,'country'] = 'Netherlands'
clustering_df_Switzerland.loc[:,'country'] = 'Switzerland'


''' To merge the two clustering_dfs'''
clustering_3_countries_df = clustering_df_Belgium.append(clustering_df_Netherlands)
clustering_3_countries_df = clustering_3_countries_df.append(clustering_df_Switzerland)


# To reset the index
clustering_3_countries_df = clustering_3_countries_df.reset_index(drop=True)
clustering_3_countries_df

In [ ]:
''' To rescale all numerical attributes of the clustering_3_countries_df '''
scaled_clustering_3_countries_df = scale_clustering_attributes_to_df(clustering_3_countries_df)
scaled_clustering_3_countries_arr = scale_clustering_attributes_to_arr(clustering_3_countries_df.drop('country', axis=1))
scaled_clustering_3_countries_df
scaled_clustering_3_countries_arr

''' To apply PCA and to go from an array of 5 dimensions to an array of 2 dimensions'''
x_pca_clustering_3_countries_arr = create_array_two_components(scaled_clustering_3_countries_arr)
x_pca_clustering_3_countries_arr

''' To creata a PCA plot from x_pca_clustering_merged'''
PCA_plot_before_clustering_3_countries = create_PCA_plot_one_argument(x_pca_clustering_3_countries_arr)
PCA_plot_before_clustering_3_countries

In [ ]:
''' To split the scaled_clustering_3_countries_df '''
scaled_clustering_df_Belgium_from_3_countries = scaled_clustering_3_countries_df[scaled_clustering_3_countries_df['country'] == 'Belgium']
scaled_clustering_df_Netherlands_from_3_countries = scaled_clustering_3_countries_df[scaled_clustering_3_countries_df['country'] == 'Netherlands']
scaled_clustering_df_Switzerland_from_3_countries = scaled_clustering_3_countries_df[scaled_clustering_3_countries_df['country'] == 'Switzerland']


scaled_clustering_df_Belgium_from_3_countries
scaled_clustering_df_Netherlands_from_3_countries
scaled_clustering_df_Switzerland_from_3_countries

''' To convert the x_pca_clustering_merged array to a df and to add the country attribute'''
# To convert the x_pca_clustering_merged array to a df
x_pca_clustering_3_countries_df = pd.DataFrame({'Principal component 1': x_pca_clustering_3_countries_arr[:, 0], 'Principal component 2': x_pca_clustering_3_countries_arr[:, 1]})

# To add the country attribute
x_pca_clustering_3_countries_df['country'] = scaled_clustering_3_countries_df['country']
x_pca_clustering_3_countries_df

''' To split the x_pca_clustering_3_countries_df df into two for both the countries '''
x_pca_clustering_df_Belgium_from_3_countries = x_pca_clustering_3_countries_df[x_pca_clustering_3_countries_df['country'] == 'Belgium']
x_pca_clustering_df_Netherlands_from_3_countries = x_pca_clustering_3_countries_df[x_pca_clustering_3_countries_df['country'] == 'Netherlands']
x_pca_clustering_df_Switzerland_from_3_countries = x_pca_clustering_3_countries_df[x_pca_clustering_3_countries_df['country'] == 'Switzerland']
x_pca_clustering_df_Belgium_from_3_countries
x_pca_clustering_df_Netherlands_from_3_countries
x_pca_clustering_df_Switzerland_from_3_countries

''' To drop the country attribute and to convert the x_pca_clustering dfs of both countries to an array'''
# To drop the country attribute
x_pca_clustering_df_Belgium_from_3_countries = x_pca_clustering_df_Belgium_from_3_countries.drop('country', axis=1)
x_pca_clustering_df_Netherlands_from_3_countries = x_pca_clustering_df_Netherlands_from_3_countries.drop('country', axis=1)
x_pca_clustering_df_Switzerland_from_3_countries = x_pca_clustering_df_Switzerland_from_3_countries.drop('country', axis=1)

# To convert the x_pca_clustering dfs of both countries to an array
x_pca_clustering_arr_Belgium_from_3_countries = x_pca_clustering_df_Belgium_from_3_countries.to_numpy()
x_pca_clustering_arr_Netherlands_from_3_countries = x_pca_clustering_df_Netherlands_from_3_countries.to_numpy()
x_pca_clustering_arr_Switzerland_from_3_countries = x_pca_clustering_df_Switzerland_from_3_countries.to_numpy()

x_pca_clustering_arr_Belgium_from_3_countries
x_pca_clustering_arr_Netherlands_from_3_countries
x_pca_clustering_arr_Switzerland_from_3_countries

In [ ]:
''' To apply the elbow method to get the optimal number of clusters of routes in each railway system'''
optimal_number_clusters_Belgium_from_3_countries = optimal_number_clusters(scaled_clustering_df_Belgium_from_3_countries.drop('country', axis=1))
optimal_number_clusters_Netherlands_from_3_countries = optimal_number_clusters(scaled_clustering_df_Netherlands_from_3_countries.drop('country', axis=1))
optimal_number_clusters_Switzerland_from_3_countries = optimal_number_clusters(scaled_clustering_df_Switzerland_from_3_countries.drop('country', axis=1))


optimal_number_clusters_Belgium_from_3_countries
optimal_number_clusters_Netherlands_from_3_countries
optimal_number_clusters_Switzerland_from_3_countries

In [ ]:
''' To assign the cluster group labels to the non_scaled_5D_df, the scaled_5D_df and the scaled_2D_df'''
km_5D_Belgium_from_3_countries, clustering_cluster_groups_5D_Belgium_from_3_countries, scaled_clustering_cluster_groups_5D_Belgium_from_3_countries, scaled_clustering_cluster_groups_2D_Belgium_from_3_countries = assign_cluster_group(optimal_number_clusters_Belgium_from_3_countries, clustering_df_Belgium.drop('country', axis=1), scaled_clustering_df_Belgium_from_3_countries.drop('country', axis=1), x_pca_clustering_df_Belgium_from_3_countries)
km_5D_Netherlands_from_3_countries, clustering_cluster_groups_5D_Netherlands_from_3_countries, scaled_clustering_cluster_groups_5D_Netherlands_from_3_countries, scaled_clustering_cluster_groups_2D_Netherlands_from_3_countries = assign_cluster_group(optimal_number_clusters_Netherlands_from_3_countries, clustering_df_Netherlands.drop('country', axis=1), scaled_clustering_df_Netherlands_from_3_countries.drop('country', axis=1), x_pca_clustering_df_Netherlands_from_3_countries)
km_5D_Switzerland_from_3_countries, clustering_cluster_groups_5D_Switzerland_from_3_countries, scaled_clustering_cluster_groups_5D_Switzerland_from_3_countries, scaled_clustering_cluster_groups_2D_Switzerland_from_3_countries = assign_cluster_group(optimal_number_clusters_Switzerland_from_3_countries, clustering_df_Switzerland.drop('country', axis=1), scaled_clustering_df_Switzerland_from_3_countries.drop('country', axis=1), x_pca_clustering_df_Switzerland_from_3_countries)


clustering_cluster_groups_5D_Belgium_from_3_countries
scaled_clustering_cluster_groups_5D_Belgium_from_3_countries
scaled_clustering_cluster_groups_2D_Belgium_from_3_countries
clustering_cluster_groups_5D_Netherlands_from_3_countries
scaled_clustering_cluster_groups_5D_Netherlands_from_3_countries
scaled_clustering_cluster_groups_2D_Netherlands_from_3_countries
clustering_cluster_groups_5D_Switzerland_from_3_countries
scaled_clustering_cluster_groups_5D_Switzerland_from_3_countries
scaled_clustering_cluster_groups_2D_Switzerland_from_3_countries

In [ ]:
''' To calculate the cluster centroids of the scaled_5D_df and the scaled_2D_df '''
cluster_centroids_5D_Belgium_from_3_countries = find_cluster_centroids_in_5D(scaled_clustering_cluster_groups_5D_Belgium_from_3_countries)
cluster_centroids_5D_Netherlands_from_3_countries = find_cluster_centroids_in_5D(scaled_clustering_cluster_groups_5D_Netherlands_from_3_countries)
cluster_centroids_5D_Switzerland_from_3_countries = find_cluster_centroids_in_5D(scaled_clustering_cluster_groups_5D_Switzerland_from_3_countries)
cluster_centroids_2D_Belgium_from_3_countries = find_cluster_centroids_in_2D(scaled_clustering_cluster_groups_2D_Belgium_from_3_countries)
cluster_centroids_2D_Netherlands_from_3_countries = find_cluster_centroids_in_2D(scaled_clustering_cluster_groups_2D_Netherlands_from_3_countries)
cluster_centroids_2D_Switzerland_from_3_countries = find_cluster_centroids_in_2D(scaled_clustering_cluster_groups_2D_Switzerland_from_3_countries)


cluster_centroids_5D_Belgium_from_3_countries
cluster_centroids_5D_Netherlands_from_3_countries
cluster_centroids_5D_Switzerland_from_3_countries
cluster_centroids_2D_Belgium_from_3_countries
cluster_centroids_2D_Netherlands_from_3_countries
cluster_centroids_2D_Switzerland_from_3_countries

# To compare the 5D centroids of the get_cluster_centroids definition to the 5D centroids of the find_cluster_centroids_in_5D definition
# These centroids values are almost equal. 
#centroidtest_5D_scaled = get_cluster_centroids(km_5D_Belgium_from_3_countries)
#centroidtest_5D_scaled

In [ ]:
''' To make the 2D PCA plot of Belgium with the Belgian cluster centroids'''
PCA_plot_clustering_Belgium_from_3_countries_belgian_centroids = create_PCA_plot(x_pca_clustering_arr_Belgium_from_3_countries, cluster_centroids_2D_Belgium_from_3_countries, scaled_clustering_cluster_groups_5D_Belgium_from_3_countries)
PCA_plot_clustering_Belgium_from_3_countries_belgian_centroids

''' To make the 2D PCA plot of Netherlands with the Dutch cluster centroids'''
PCA_plot_clustering_Netherlands_from_3_countries_dutch_centroids = create_PCA_plot(x_pca_clustering_arr_Netherlands_from_3_countries, cluster_centroids_2D_Netherlands_from_3_countries, scaled_clustering_cluster_groups_5D_Netherlands_from_3_countries)
PCA_plot_clustering_Netherlands_from_3_countries_dutch_centroids

''' To make the 2D PCA plot of Switzerland with the Swiss cluster centroids'''
PCA_plot_clustering_Switzerland_from_3_countries_swiss_centroids = create_PCA_plot(x_pca_clustering_arr_Switzerland_from_3_countries, cluster_centroids_2D_Switzerland_from_3_countries, scaled_clustering_cluster_groups_5D_Switzerland_from_3_countries)
PCA_plot_clustering_Switzerland_from_3_countries_swiss_centroids

In [ ]:
''' To make the 2D PCA plot of Belgium with the Dutch cluster centroids'''
PCA_plot_clustering_Belgium_from_3_countries_dutch_centroids = create_PCA_plot(x_pca_clustering_arr_Belgium_from_3_countries, cluster_centroids_2D_Netherlands_from_3_countries, scaled_clustering_cluster_groups_5D_Belgium_from_3_countries)
PCA_plot_clustering_Belgium_from_3_countries_dutch_centroids

''' To make the 2D PCA plot of Netherlands with the Belgian cluster centroids'''
PCA_plot_clustering_Netherlands_from_3_countries_belgian_centroids = create_PCA_plot(x_pca_clustering_arr_Netherlands_from_3_countries, cluster_centroids_2D_Belgium_from_3_countries, scaled_clustering_cluster_groups_5D_Netherlands_from_3_countries)
PCA_plot_clustering_Netherlands_from_3_countries_belgian_centroids

In [ ]:
''' To make the 2D PCA plot of Belgium with the Swiss cluster centroids'''
PCA_plot_clustering_Belgium_from_3_countries_swiss_centroids = create_PCA_plot(x_pca_clustering_arr_Belgium_from_3_countries, cluster_centroids_2D_Switzerland_from_3_countries, scaled_clustering_cluster_groups_5D_Belgium_from_3_countries)
PCA_plot_clustering_Belgium_from_3_countries_swiss_centroids

''' To make the 2D PCA plot of Switzerland with the Belgian cluster centroids'''
PCA_plot_clustering_Switzerland_from_3_countries_belgian_centroids = create_PCA_plot(x_pca_clustering_arr_Switzerland_from_3_countries, cluster_centroids_2D_Belgium_from_3_countries, scaled_clustering_cluster_groups_5D_Switzerland_from_3_countries)
PCA_plot_clustering_Switzerland_from_3_countries_belgian_centroids

In [ ]:
''' To make the 2D PCA plot of Netherlands with the Swiss cluster centroids'''
PCA_plot_clustering_Netherlands_from_3_countries_swiss_centroids = create_PCA_plot(x_pca_clustering_arr_Netherlands_from_3_countries, cluster_centroids_2D_Switzerland_from_3_countries, scaled_clustering_cluster_groups_5D_Netherlands_from_3_countries)
PCA_plot_clustering_Netherlands_from_3_countries_swiss_centroids

''' To make the 2D PCA plot of Switzerland with the Dutch cluster centroids'''
PCA_plot_clustering_Switzerland_from_3_countries_dutch_centroids = create_PCA_plot(x_pca_clustering_arr_Switzerland_from_3_countries, cluster_centroids_2D_Netherlands_from_3_countries, scaled_clustering_cluster_groups_5D_Switzerland_from_3_countries)
PCA_plot_clustering_Switzerland_from_3_countries_dutch_centroids

In [ ]:
cluster_centroids_2D_Netherlands_from_3_countries

## K-nearest neighbor method

### Functions

In [ ]:
''' To drop the country attribute from the scaled_clustering_df and to convert the result to an array'''
def get_array_routes_other_country(scaled_clustering_df):
    scaled_clustering_arr = scaled_clustering_df.drop('country', axis=1)
    scaled_clustering_arr = scaled_clustering_arr.to_numpy()

    return scaled_clustering_arr

In [ ]:
'''To split the first centroid in the cluster_centroids_5D array off from the other centroids in that array '''
def split_off_the_first_centroid(cluster_centroids_5D):
    first_centroid = cluster_centroids_5D[:1]
    other_centroids = cluster_centroids_5D[1:]

    return first_centroid, other_centroids

In [ ]:
''' To get the array that contains the k nearest neighbors of one centroid'''
def get_k_nearest_neighbors_of_centroid(centroid_routes, nearest, k):
    for j in nearest[:, 1:k+1]:
        return centroid_routes[j]

In [ ]:
''' To get the list of indices that correspond to the indices of the k nearest neighbors of one centroid in the route_characteristics df'''
def get_indices_k_nearest_neighbors_of_centroid(nearest, k):
    list_indices = []
    for i in nearest[0, 1:k+1]:
        list_indices.append(i-1)

    return list_indices

In [ ]:
''' To create the nearest array that contains for each array element an index that corresponds to an array element index of the centroid_routes_arr together 
with the k-nearest neighbor indices of the array element indices of the centroid_routes_arr'''
def to_get_k_nearest_neighbors_of_centroid(centroid_routes_arr):
    D = distance.squareform(distance.pdist(centroid_routes_arr))
    nearest = np.argsort(D, axis=1)

    nearest_neighbors_centroid = get_k_nearest_neighbors_of_centroid(centroid_routes_arr, nearest, 5)
    list_indices_nearest_neighbors = get_indices_k_nearest_neighbors_of_centroid(nearest, 5)

    return nearest_neighbors_centroid, list_indices_nearest_neighbors

In [ ]:
''' To concatenate the array that contains one centroid with the array that contains the scaled route characteristics of the routes of another country'''
def to_get_array_centroid_routes(first_centroid, scaled_clustering_arr):
    centroid_routes_arr = np.concatenate((first_centroid, scaled_clustering_arr), axis=0)

    return centroid_routes_arr

In [ ]:
''' To activate the definition that add the cluster centroid as the first array element of the array that contains the other routes and
to activate the definitions to get the k-nearest neighbors of a centroid as well as the indices of the nearest neighbors in the route_characteristics df'''
def find_k_nearest_neighbors_of_one_centroid(first_centroid, scaled_clustering_arr):
    centroid_routes_arr = to_get_array_centroid_routes(first_centroid, scaled_clustering_arr)
    nearest_neighbors_centroid, list_indices_nearest_neighbors = to_get_k_nearest_neighbors_of_centroid(centroid_routes_arr)

    return nearest_neighbors_centroid, list_indices_nearest_neighbors

In [ ]:
''' To find the nearest_neighbors of all centroids and to find the indices of these nearest_neighbors'''
def find_k_nearest_neighbors_of_all_centroids(cluster_centroids_5D, scaled_clustering_df):
    centroids_list = []
    nearest_neighbors_list = []
    indices_nearest_neighbors_list = []
    number_of_cluster_centroids = len(cluster_centroids_5D)
    print("Number of centroids:", number_of_cluster_centroids)
    other_centroids = cluster_centroids_5D.copy()
    scaled_clustering_arr = get_array_routes_other_country(scaled_clustering_df)
    for i in range(0, number_of_cluster_centroids):
        centroid, other_centroids = split_off_the_first_centroid(other_centroids)
        nearest_neighbors_centroid, list_indices_nearest_neighbors = find_k_nearest_neighbors_of_one_centroid(centroid, scaled_clustering_arr)
        nearest_neighbors_list.append(nearest_neighbors_centroid)
        indices_nearest_neighbors_list.append(list_indices_nearest_neighbors)
        centroids_list.append(centroid)    

    return nearest_neighbors_list, indices_nearest_neighbors_list, centroids_list

In [ ]:
''' To find the nearest neighbors of all centroids and their corresponding route_ids'''
def search_k_nearest_neighbors_of_centroids(cluster_centroids_5D, scaled_clustering_df, route_characteristics):
    nearest_neighbors_list, indices_nearest_neighbors_list, centroids_list = find_k_nearest_neighbors_of_all_centroids(cluster_centroids_5D, scaled_clustering_df)
    k_nearest_neighbors_df = pd.DataFrame(index=range(len(centroids_list)),columns=['centroid', 'nearest_neighbors_centroid', 'route_ids_nearest_neighbors'])
    for nr_centroid, centroid in enumerate(centroids_list):
        #[0] is used to only get one array and not an array of an array
        k_nearest_neighbors_df.loc[nr_centroid,'centroid'] = centroid[0]
        k_nearest_neighbors_df.loc[nr_centroid, 'nearest_neighbors_centroid'] = nearest_neighbors_list[nr_centroid]
        sublist = indices_nearest_neighbors_list[nr_centroid]
        list_route_ids = []
        for listelement in sublist:
            route_id = route_characteristics.loc[listelement,'route_id']
            list_route_ids.append(route_id)
            k_nearest_neighbors_df.loc[nr_centroid, 'route_ids_nearest_neighbors'] = list_route_ids

    return k_nearest_neighbors_df

### Belgian centroids and Dutch routes

In [ ]:
k_nearest_neighbors_df_belgian_centroids_dutch_routes = search_k_nearest_neighbors_of_centroids(cluster_centroids_5D_Belgium_from_3_countries, scaled_clustering_df_Netherlands_from_3_countries, route_characteristics_Netherlands)
k_nearest_neighbors_df_belgian_centroids_dutch_routes

### Dutch centroids and Belgian routes

In [ ]:
k_nearest_neighbors_df_dutch_centroids_belgian_routes = search_k_nearest_neighbors_of_centroids(cluster_centroids_5D_Netherlands_from_3_countries, scaled_clustering_df_Belgium_from_3_countries, route_characteristics_Belgium)
k_nearest_neighbors_df_dutch_centroids_belgian_routes

### Belgian centroids and Swiss routes

In [ ]:
k_nearest_neighbors_df_belgian_centroids_swiss_routes = search_k_nearest_neighbors_of_centroids(cluster_centroids_5D_Belgium_from_3_countries, scaled_clustering_df_Switzerland_from_3_countries, route_characteristics_Switzerland)
k_nearest_neighbors_df_belgian_centroids_swiss_routes

### Swiss centroids and Belgian routes

In [ ]:
k_nearest_neighbors_df_swiss_centroids_belgian_routes = search_k_nearest_neighbors_of_centroids(cluster_centroids_5D_Switzerland_from_3_countries, scaled_clustering_df_Belgium_from_3_countries, route_characteristics_Belgium)
k_nearest_neighbors_df_swiss_centroids_belgian_routes

### Dutch centroids and Swiss routes

In [ ]:
k_nearest_neighbors_df_dutch_centroids_swiss_routes = search_k_nearest_neighbors_of_centroids(cluster_centroids_5D_Netherlands_from_3_countries, scaled_clustering_df_Switzerland_from_3_countries, route_characteristics_Switzerland)
k_nearest_neighbors_df_dutch_centroids_swiss_routes

### Swiss centroids and dutch routes

In [ ]:
k_nearest_neighbors_df_swiss_centroids_dutch_routes = search_k_nearest_neighbors_of_centroids(cluster_centroids_5D_Switzerland_from_3_countries, scaled_clustering_df_Netherlands_from_3_countries, route_characteristics_Netherlands)
k_nearest_neighbors_df_swiss_centroids_dutch_routes

### Belgian centroids and belgian routes


In [ ]:
k_nearest_neighbors_df_belgian_centroids_belgian_routes = search_k_nearest_neighbors_of_centroids(cluster_centroids_5D_Belgium_from_3_countries, scaled_clustering_df_Belgium_from_3_countries, route_characteristics_Belgium)
k_nearest_neighbors_df_belgian_centroids_belgian_routes

### Dutch centroids and dutch routes


In [ ]:
k_nearest_neighbors_df_dutch_centroids_dutch_routes = search_k_nearest_neighbors_of_centroids(cluster_centroids_5D_Netherlands_from_3_countries, scaled_clustering_df_Netherlands_from_3_countries, route_characteristics_Netherlands)
k_nearest_neighbors_df_dutch_centroids_dutch_routes

### Swiss centroids and swiss routes


In [ ]:
k_nearest_neighbors_df_swiss_centroids_swiss_routes = search_k_nearest_neighbors_of_centroids(cluster_centroids_5D_Switzerland_from_3_countries, scaled_clustering_df_Switzerland_from_3_countries, route_characteristics_Switzerland)
k_nearest_neighbors_df_swiss_centroids_swiss_routes

## Travel time comparison

### Functions 

In [ ]:
''' To add a new avg_travel_time_centroid attribute to the df'''
def add_avg_travel_time_centroid(scaled_clustering_cluster_groups_df, route_characteristics, routes_country, k_nearest_neighbors_df):
    # To add the route_id attribute from the route_characteristics_df to the scaled_clustering_cluster_groups_5D_df
    travel_time_clustering_cluster_groups_df = scaled_clustering_cluster_groups_df.copy()
    travel_time_clustering_cluster_groups_df = travel_time_clustering_cluster_groups_df.reset_index(drop=True)
    travel_time_clustering_cluster_groups_df['route_id'] = route_characteristics['route_id']

    # To merge the country_routes df with the travel_time_clustering_cluster_groups_df on route_id to get the average travel time of a route
    travel_time_clustering_cluster_groups_df = pd.merge(routes_country[['route_id', 'travel_time']], travel_time_clustering_cluster_groups_df, on='route_id')

    # To drop the route_id attribute
    travel_time_clustering_cluster_groups_df = travel_time_clustering_cluster_groups_df.drop('route_id', axis=1)

    # To search for the centroid values of the travel_time for each cluster
    travel_time_centroids_arr = find_cluster_centroids_in_5D(travel_time_clustering_cluster_groups_df)

    # To add the average travel times of the centroids in a list
    list_avg_travel_time_centroids = []
    travel_time_centroids_list = travel_time_centroids_arr.tolist()
    for sublist in travel_time_centroids_list:
        avg_travel_time_centroid = sublist[0]
        list_avg_travel_time_centroids.append(avg_travel_time_centroid)

    # To add the list_avg_travel_time_centroids as a new attribute of the df
    travel_time_df = k_nearest_neighbors_df.copy()
    travel_time_df['avg_travel_time_centroid'] = list_avg_travel_time_centroids

    return travel_time_df

In [ ]:
''' To add a new travel_time_nearest_neighbors_centroid attribute to the df'''
def add_travel_time_nearest_neighbors_centroid(travel_time_df_1, routes_country):
    # To copy the travel_time_df_1 and to add a new attribute avg_travel_time_nearest_neighbors_centroid
    travel_time_df = travel_time_df_1.copy()
    travel_time_df['travel_time_nearest_neighbors_centroid'] = ""
    travel_time_df['waiting_time_nearest_neighbors_centroid'] = ""

    # To loop over the row indices of the travel_time_df
    for row in range(len(travel_time_df)):
        # To select the route_ids_nearest_neighbors attribute value of that row
        list_route_ids_nearest_neighbors = travel_time_df.loc[row, 'route_ids_nearest_neighbors']
        # To create an empty list list_avg_travel_time_nearest_neighbors_centroid the same is done for waiting time
        list_travel_time_nearest_neighbors_centroid = []
        list_waiting_time_nearest_neighbors_centroid = []
        # To loop over all elements in the list_route_ids_nearest_neighbors
        for route_id in list_route_ids_nearest_neighbors:
            # To search for the corresponding travel time of this route_id
            travel_time = routes_country.loc[routes_country['route_id'] == route_id, 'travel_time'].iloc[0]
            # To search for the corresponding waiting time of this route_id
            waiting_time = routes_country.loc[routes_country['route_id'] == route_id, 'waiting_time'].iloc[0]
            # To add the corresponding average travel time and waiting time of the route to the list_avg_travel_time_nearest_neighbors_centroids
            list_travel_time_nearest_neighbors_centroid.append(travel_time)
            list_waiting_time_nearest_neighbors_centroid.append(waiting_time)
        # To add the list_travel_time_nearest_neighbors_centroid as a new value in the avg_travel_time_nearest_neighbors_centroid column the same is done for waiting time
        travel_time_df.at[row, 'travel_time_nearest_neighbors_centroid'] = list_travel_time_nearest_neighbors_centroid
        travel_time_df.at[row, 'waiting_time_nearest_neighbors_centroid'] = list_waiting_time_nearest_neighbors_centroid


    return travel_time_df

In [ ]:
def get_the_average_travel_time(scaled_clustering_cluster_groups_df, route_characteristics, k_nearest_neighbors_df, routes_centroid_country, routes_non_centroid_country):
    travel_time_df_1 = add_avg_travel_time_centroid(scaled_clustering_cluster_groups_df, route_characteristics, routes_centroid_country, k_nearest_neighbors_df)
    travel_time_df = add_travel_time_nearest_neighbors_centroid(travel_time_df_1, routes_non_centroid_country)

    return travel_time_df

In [ ]:
def calculate_std_tt_nn_centroid(travel_time_df):
    #calculates variance of the travel_time_nearest_neighbors_centroid
    travel_time_df['std_tt_nn_centroid'] = travel_time_df['travel_time_nearest_neighbors_centroid'].apply(lambda x: np.std(x))
    return travel_time_df

In [ ]:
def calculate_avg_tt_nn_centroid(travel_time_df):
    #calculates average of the travel_time_nearest_neighbors_centroid
    travel_time_df['avg_tt_nn_centroid'] = travel_time_df['travel_time_nearest_neighbors_centroid'].apply(lambda x: np.mean(x))
    return travel_time_df

### Belgian centroids and dutch routes

In [ ]:
travel_time_df_belgian_centroids_dutch_routes = get_the_average_travel_time(scaled_clustering_cluster_groups_5D_Belgium_from_3_countries, route_characteristics_Belgium, k_nearest_neighbors_df_belgian_centroids_dutch_routes, belgian_routes_Belgium, dutch_routes_Netherlands)
travel_time_df_belgian_centroids_dutch_routes = calculate_std_tt_nn_centroid(travel_time_df_belgian_centroids_dutch_routes)
travel_time_df_belgian_centroids_dutch_routes = calculate_avg_tt_nn_centroid(travel_time_df_belgian_centroids_dutch_routes)
travel_time_df_belgian_centroids_dutch_routes

### Dutch centroids and belgian routes

In [ ]:
travel_time_df_dutch_centroids_belgian_routes = get_the_average_travel_time(scaled_clustering_cluster_groups_5D_Netherlands_from_3_countries, route_characteristics_Netherlands, k_nearest_neighbors_df_dutch_centroids_belgian_routes, dutch_routes_Netherlands, belgian_routes_Belgium)
travel_time_df_dutch_centroids_belgian_routes = calculate_std_tt_nn_centroid(travel_time_df_dutch_centroids_belgian_routes)
travel_time_df_dutch_centroids_belgian_routes = calculate_avg_tt_nn_centroid(travel_time_df_dutch_centroids_belgian_routes)
travel_time_df_dutch_centroids_belgian_routes

### Belgian centroids and Swiss routes

In [ ]:
travel_time_df_belgian_centroids_swiss_routes = get_the_average_travel_time(scaled_clustering_cluster_groups_5D_Belgium_from_3_countries, route_characteristics_Belgium, k_nearest_neighbors_df_belgian_centroids_swiss_routes, belgian_routes_Belgium, swiss_routes_Switzerland)
travel_time_df_belgian_centroids_swiss_routes = calculate_std_tt_nn_centroid(travel_time_df_belgian_centroids_swiss_routes)
travel_time_df_belgian_centroids_swiss_routes = calculate_avg_tt_nn_centroid(travel_time_df_belgian_centroids_swiss_routes)
travel_time_df_belgian_centroids_swiss_routes

### Swiss centroids and Belgian routes

In [ ]:
travel_time_df_swiss_centroids_belgian_routes = get_the_average_travel_time(scaled_clustering_cluster_groups_5D_Switzerland_from_3_countries, route_characteristics_Switzerland, k_nearest_neighbors_df_swiss_centroids_belgian_routes, swiss_routes_Switzerland, belgian_routes_Belgium)
travel_time_df_swiss_centroids_belgian_routes = calculate_std_tt_nn_centroid(travel_time_df_swiss_centroids_belgian_routes)
travel_time_df_swiss_centroids_belgian_routes = calculate_avg_tt_nn_centroid(travel_time_df_swiss_centroids_belgian_routes)
travel_time_df_swiss_centroids_belgian_routes

### Dutch centroids and Swiss routes

In [ ]:
travel_time_df_dutch_centroids_swiss_routes = get_the_average_travel_time(scaled_clustering_cluster_groups_5D_Netherlands_from_3_countries, route_characteristics_Netherlands, k_nearest_neighbors_df_dutch_centroids_swiss_routes, dutch_routes_Netherlands, swiss_routes_Switzerland)
travel_time_df_dutch_centroids_swiss_routes = calculate_std_tt_nn_centroid(travel_time_df_dutch_centroids_swiss_routes)
travel_time_df_dutch_centroids_swiss_routes = calculate_avg_tt_nn_centroid(travel_time_df_dutch_centroids_swiss_routes)
travel_time_df_dutch_centroids_swiss_routes

### Swiss centroids and dutch routes

In [ ]:
travel_time_df_swiss_centroids_dutch_routes = get_the_average_travel_time(scaled_clustering_cluster_groups_5D_Switzerland_from_3_countries, route_characteristics_Switzerland, k_nearest_neighbors_df_swiss_centroids_dutch_routes, swiss_routes_Switzerland, dutch_routes_Netherlands)
travel_time_df_swiss_centroids_dutch_routes = calculate_std_tt_nn_centroid(travel_time_df_swiss_centroids_dutch_routes)
travel_time_df_swiss_centroids_dutch_routes = calculate_avg_tt_nn_centroid(travel_time_df_swiss_centroids_dutch_routes)
travel_time_df_swiss_centroids_dutch_routes

### Belgian centroids with Belgian routes

In [ ]:
travel_time_df_belgian_centroids_belgian_routes = get_the_average_travel_time(scaled_clustering_cluster_groups_5D_Belgium_from_3_countries, route_characteristics_Belgium, k_nearest_neighbors_df_belgian_centroids_belgian_routes, belgian_routes_Belgium, belgian_routes_Belgium)
travel_time_df_belgian_centroids_belgian_routes = calculate_std_tt_nn_centroid(travel_time_df_belgian_centroids_belgian_routes)
travel_time_df_belgian_centroids_belgian_routes = calculate_avg_tt_nn_centroid(travel_time_df_belgian_centroids_belgian_routes)
travel_time_df_belgian_centroids_belgian_routes

### Dutch centroids with Dutch routes

In [ ]:
travel_time_df_dutch_centroids_dutch_routes = get_the_average_travel_time(scaled_clustering_cluster_groups_5D_Netherlands_from_3_countries, route_characteristics_Netherlands, k_nearest_neighbors_df_dutch_centroids_dutch_routes, dutch_routes_Netherlands, dutch_routes_Netherlands)
travel_time_df_dutch_centroids_dutch_routes = calculate_std_tt_nn_centroid(travel_time_df_dutch_centroids_dutch_routes)
travel_time_df_dutch_centroids_dutch_routes = calculate_avg_tt_nn_centroid(travel_time_df_dutch_centroids_dutch_routes)
travel_time_df_dutch_centroids_dutch_routes

### Swiss centroids with swiss routes

In [ ]:
travel_time_df_swiss_centroids_swiss_routes = get_the_average_travel_time(scaled_clustering_cluster_groups_5D_Switzerland_from_3_countries, route_characteristics_Switzerland, k_nearest_neighbors_df_swiss_centroids_swiss_routes, swiss_routes_Switzerland, swiss_routes_Switzerland)
travel_time_df_swiss_centroids_swiss_routes = calculate_std_tt_nn_centroid(travel_time_df_swiss_centroids_swiss_routes)
travel_time_df_swiss_centroids_swiss_routes = calculate_avg_tt_nn_centroid(travel_time_df_swiss_centroids_swiss_routes)
travel_time_df_swiss_centroids_swiss_routes

### Merge the important information and add avg abs difference nn1 with nn2

#### Functions

In [ ]:
'''Change the column names mentionned in important columns by adding the suffix at the end and return those new names in a lists (new_important columns)'''

def change_col_name_with_suffix(travel_time_df, important_columns, suffix):
    new_important_columns = []
    for name_col in important_columns:
        new_name = name_col + suffix
        travel_time_df = travel_time_df.rename(columns={name_col: new_name})
        new_important_columns.append(new_name)
    return travel_time_df , new_important_columns

In [ ]:
'''Merge the important information (avg, differences and variance) of the centroids with the k-nearest neighbours'''

def merge_important_infomation_centroid_nn(travel_time_df_main_centroids_main_routes, travel_time_df_main_centroids_other1_routes, travel_time_df_main_centroids_other2_routes, main_suffix, other1_suffix, other2_suffix):
    important_columns = ['nearest_neighbors_centroid', 'travel_time_nearest_neighbors_centroid', 'route_ids_nearest_neighbors', 'std_tt_nn_centroid', 'avg_tt_nn_centroid']
    additional_columns_main = ['centroid', 'avg_travel_time_centroid']
    
    #change the names of the important columns by adding their suffix

    travel_time_df_main_centroids_main_routes, important_columns_main = change_col_name_with_suffix(travel_time_df_main_centroids_main_routes, important_columns, main_suffix)
    travel_time_df_main_centroids_other1_routes, important_columns_other1 = change_col_name_with_suffix(travel_time_df_main_centroids_other1_routes, important_columns, other1_suffix)
    travel_time_df_main_centroids_other2_routes, important_columns_other2 = change_col_name_with_suffix(travel_time_df_main_centroids_other2_routes, important_columns, other2_suffix)
   
    #make the merges between all the DataFrames
    important_information_centroid_nn = pd.merge(travel_time_df_main_centroids_main_routes[additional_columns_main + important_columns_main ], travel_time_df_main_centroids_other1_routes[important_columns_other1 + ['centroid']] ,left_index = True, right_index = True,  on='centroid')
    important_information_centroid_nn = pd.merge(important_information_centroid_nn, travel_time_df_main_centroids_other2_routes[important_columns_other2 + ['centroid']] ,left_index = True, right_index = True,  on='centroid')
    
    return important_information_centroid_nn

In [ ]:
'''Calculates the avg absolute difference between the nearest neighbors of the centroid's country and the nearest neighbors of the other countries'''

def calculate_avg_abs_difference_tt_nn1_tt_nn2(important_information_df):
    countries = find_countries_name_important_information_df(important_information_df)
    important_information_df[f'avg_abs_difference_tt_nn_{countries[0]}_tt_nn_{countries[1]}'] = np.nan
    important_information_df[f'avg_abs_difference_tt_nn_{countries[0]}_tt_nn_{countries[2]}'] = np.nan
    for index_row, row in important_information_df.iterrows():
        #instantiate the list containing the avg abs differences of each centroid and the nearest neighbors of country one for 1 and two for 2
        list_avg_abs_diff_1 = []
        list_avg_abs_diff_2 = []
        #loop over the tt of the nearest neighbors of the centroid's country
        for tt_nn_centroid_country in row[f'travel_time_nearest_neighbors_centroid_{countries[0]}']:
            #instatiate the list containg the abs differences of one centroid and the nearest neighbors of a country
            list_abs_diff = []
            #loop over the tt of the nearest neighbors of other first country
            for tt_nn_country1 in row[f'travel_time_nearest_neighbors_centroid_{countries[1]}']:
                #calculate the abs difference for each nn_country1 (with the nn of the centroid's country)
                list_abs_diff.append(abs(tt_nn_centroid_country - tt_nn_country1))
            list_avg_abs_diff_1.append(statistics.mean(list_abs_diff))
            #instatiate the list containg the abs differences of one centroid and the nearest neighbors of a country
            list_abs_diff = []
            #loop over the tt of the nearest neighbors of other second country
            for tt_nn_country2 in row[f'travel_time_nearest_neighbors_centroid_{countries[2]}']:
                #calculate the abs difference for each nn_country2 (with the nn of the centroid's country)
                list_abs_diff.append(abs(tt_nn_centroid_country - tt_nn_country2))
            list_avg_abs_diff_2.append(statistics.mean(list_abs_diff))
        #adds the avg abs differences of the centroid's country nn and the country's 1 nearest neighbors
        important_information_df.loc[index_row, f'avg_abs_difference_tt_nn_{countries[0]}_tt_nn_{countries[1]}'] = statistics.mean(list_avg_abs_diff_1)
        #adds the avg abs differences of the centroid's country nn and the country's 2 nearest neighbors
        important_information_df.loc[index_row, f'avg_abs_difference_tt_nn_{countries[0]}_tt_nn_{countries[2]}'] = statistics.mean(list_avg_abs_diff_2)
    return important_information_df

#### Acutal merging

In [ ]:
'''Analysis unsing Belgian centroids'''
comparison_table_Belgium = merge_important_infomation_centroid_nn(travel_time_df_belgian_centroids_belgian_routes, travel_time_df_belgian_centroids_dutch_routes, travel_time_df_belgian_centroids_swiss_routes, '_Belgium', '_Netherlands', '_Switzerland')
comparison_table_Belgium = calculate_avg_abs_difference_tt_nn1_tt_nn2(comparison_table_Belgium)
comparison_table_Belgium

In [ ]:
'''Analysis unsing Dutch centroids'''
comparison_table_Netherlands = merge_important_infomation_centroid_nn(travel_time_df_dutch_centroids_dutch_routes, travel_time_df_dutch_centroids_belgian_routes, travel_time_df_dutch_centroids_swiss_routes, '_Netherlands', '_Belgium', '_Switzerland')
comparison_table_Netherlands = calculate_avg_abs_difference_tt_nn1_tt_nn2(comparison_table_Netherlands)
comparison_table_Netherlands

In [ ]:
'''Analysis unsing Swiss centroids'''
comparison_table_Switzerland = merge_important_infomation_centroid_nn(travel_time_df_swiss_centroids_swiss_routes, travel_time_df_swiss_centroids_belgian_routes, travel_time_df_swiss_centroids_dutch_routes,'_Switzerland', '_Belgium', '_Netherlands')
comparison_table_Switzerland = calculate_avg_abs_difference_tt_nn1_tt_nn2(comparison_table_Switzerland)
comparison_table_Switzerland

## Statistical analysis of the results 

In [ ]:
columns_for_comparison = ['route_id', 'nodes','max_degree', 'max_closeness_centrality', 'max_betweenness_centrality','len_stop_sequence', 'absolute_distance', 'travel_time']

In [ ]:
'''Create the filtered DataFrames for our analysis'''

columns_analysis_std_tt = ['std_tt_nn_centroid_Belgium', 'std_tt_nn_centroid_Netherlands', 'std_tt_nn_centroid_Switzerland']
columns_analysis_avg_tt = ['avg_tt_nn_centroid_Belgium', 'avg_tt_nn_centroid_Netherlands', 'avg_tt_nn_centroid_Switzerland']
columns_analysis_avgtt = ['centroid', 'avg_travel_time_centroid','travel_time_nearest_neighbors_centroid_Belgium', 'travel_time_nearest_neighbors_centroid_Netherlands', 'travel_time_nearest_neighbors_centroid_Switzerland']
columns_analysis_route_id = ['centroid', 'route_ids_nearest_neighbors_Belgium', 'route_ids_nearest_neighbors_Netherlands', 'route_ids_nearest_neighbors_Switzerland']

#create the filtered dataframes
std_nn_belgian_centroid = comparison_table_Belgium[columns_analysis_std_tt]
std_nn_dutch_centroid = comparison_table_Netherlands[columns_analysis_std_tt]
std_nn_swiss_centroid = comparison_table_Switzerland[columns_analysis_std_tt]
avg_nn_belgian_centroid = comparison_table_Belgium[columns_analysis_avg_tt]
avg_nn_dutch_centroid = comparison_table_Netherlands[columns_analysis_avg_tt]
avg_nn_swiss_centroid = comparison_table_Switzerland[columns_analysis_avg_tt]
tt_nn_belgian_centroid = comparison_table_Belgium[columns_analysis_avgtt]
tt_nn_dutch_centroid = comparison_table_Netherlands[columns_analysis_avgtt]
tt_nn_swiss_centroid = comparison_table_Switzerland[columns_analysis_avgtt]
route_id_nn_belgian_centroid = comparison_table_Belgium[columns_analysis_route_id]
route_id_nn_dutch_centroid = comparison_table_Netherlands[columns_analysis_route_id]
route_id_nn_swiss_centroid = comparison_table_Switzerland[columns_analysis_route_id]
avg_abs_diff_tt_nn_belgian_centroid = comparison_table_Belgium[['avg_abs_difference_tt_nn_Belgium_tt_nn_Netherlands', 'avg_abs_difference_tt_nn_Belgium_tt_nn_Switzerland']]
avg_abs_diff_tt_nn_dutch_centroid = comparison_table_Netherlands[['avg_abs_difference_tt_nn_Netherlands_tt_nn_Belgium', 'avg_abs_difference_tt_nn_Netherlands_tt_nn_Switzerland']]
avg_abs_diff_tt_nn_swiss_centroid = comparison_table_Switzerland[['avg_abs_difference_tt_nn_Switzerland_tt_nn_Belgium', 'avg_abs_difference_tt_nn_Switzerland_tt_nn_Netherlands']]


In [ ]:
'''Set the display information to allow better screenshots'''
pd.options.display.max_rows
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
#Show only two digits after the comma
pd.options.display.float_format = "{:.2f}".format

### Functions

In [ ]:
'''Retrieve the 5 NN information in the respective route_characteristics DataFrame'''

def retrieve_5_nn_info(centroid_index, country_centroid, country_5_nn):
    if country_centroid == 'Belgium':
        df_centroid = comparison_table_Belgium
    if country_centroid == 'Netherlands':
        df_centroid = comparison_table_Netherlands
    if country_centroid == 'Switzerland':
        df_centroid = comparison_table_Switzerland
    route_id_nn = df_centroid[f'route_ids_nearest_neighbors_{country_5_nn}'].loc[centroid_index]
    if country_5_nn == 'Belgium':
        charac_nn = route_characteristics_Belgium[columns_for_comparison]
    if country_5_nn == 'Netherlands':
        charac_nn = route_characteristics_Netherlands[columns_for_comparison]
    if country_5_nn == 'Switzerland':
        charac_nn = route_characteristics_Switzerland[columns_for_comparison]
        
    return charac_nn[(charac_nn['route_id'] == route_id_nn[0])|(charac_nn['route_id'] == route_id_nn[1])|(charac_nn['route_id'] == route_id_nn[2])|(charac_nn['route_id'] == route_id_nn[3])|(charac_nn['route_id'] == route_id_nn[4])]

In [ ]:
'''Get the 2D coordinates of the centroids of the chosen country'''

def get_centroid_2D_coord(cluster_centroids_2D):
    df_centroids_2D = pd.DataFrame(columns=['First Principle Component', 'Second Principle Component'])
    for index, row in enumerate(cluster_centroids_2D):
        df_centroids_2D.loc[index, 'First Principle Component'] = row[0]
        df_centroids_2D.loc[index, 'Second Principle Component'] = row[1]
    return df_centroids_2D

In [ ]:
'''The first name of the list is the country with the centroids, the other ones are the ones of the other countries'''

def find_countries_name_important_information_df(important_information_df):
    countries = []
    for column_name in important_information_df.columns:
        if 'route_ids_nearest_neighbors' in column_name:
            countries.append(column_name.split('_')[-1])
    return countries

### Method 1: analysis of the mean and standard deviation of the nearest neighbors of a referance country

In [ ]:
std_nn_belgian_centroid

In [ ]:
std_nn_dutch_centroid.iloc[:,[0,1]]

In [ ]:
std_nn_swiss_centroid.iloc[:,[0,2]]

In [ ]:
avg_nn_belgian_centroid

In [ ]:
avg_nn_dutch_centroid.iloc[:,[0,1]]

In [ ]:
avg_nn_swiss_centroid.iloc[:,[0,2]]

### Method 2 analysis of the average abs differences (of the 25 combinations for each centroid)

#### Belgium

In [ ]:
avg_abs_diff_tt_nn_belgian_centroid.rename(columns={'avg_abs_difference_tt_nn_Belgium_tt_nn_Netherlands': 'Netherlands', 'avg_abs_difference_tt_nn_Belgium_tt_nn_Switzerland': 'Switzerland'})

In [ ]:
''' Analysis for Dutch routes with Belgian centroids'''
#Index 1
print('the Belgian centroids with the Belgian routes')
retrieve_5_nn_info(1, 'Belgium', 'Belgium')
print('the Belgian centroids with the Dutch routes')
retrieve_5_nn_info(1, 'Belgium', 'Netherlands')
    
#Index 4
print('the Belgian centroids with the Belgian routes')
retrieve_5_nn_info(4, 'Belgium', 'Belgium')
print('the Belgian centroids with the Dutch routes')
retrieve_5_nn_info(4, 'Belgium', 'Netherlands')


In [ ]:
''' Analysis for Swiss routes with Belgian centroids'''
#Index 1
print('the Belgian centroids with the Belgian routes')
retrieve_5_nn_info(1, 'Belgium', 'Belgium')
print('the Belgian centroids with the Swiss routes')
retrieve_5_nn_info(1, 'Belgium', 'Switzerland')
    
#Index 4
print('the Belgian centroids with the Belgian routes')
retrieve_5_nn_info(4, 'Belgium', 'Belgium')
print('the Belgian centroids with the Swiss routes')
retrieve_5_nn_info(4, 'Belgium', 'Switzerland')


#### Netherlands

In [ ]:
avg_abs_diff_tt_nn_dutch_centroid.rename(columns={'avg_abs_difference_tt_nn_Netherlands_tt_nn_Belgium': 'Belgium', 'avg_abs_difference_tt_nn_Netherlands_tt_nn_Switzerland': 'Switzerland'})

In [ ]:
''' Analysis for Belgian routes with Ducth centroids'''
#Index 2
print('the Dutch centroids with the Dutch routes')
retrieve_5_nn_info(2, 'Netherlands', 'Netherlands')
print('the Dutch centroids with the Belgian routes')
retrieve_5_nn_info(2, 'Netherlands', 'Belgium')
    

In [ ]:
''' Analysis for Swiss routes with Ducth centroids'''
#Index 0
print('the Dutch centroids with the Dutch routes')
retrieve_5_nn_info(0, 'Netherlands', 'Netherlands')
print('the Dutch centroids with the Swiss routes')
retrieve_5_nn_info(0, 'Netherlands', 'Switzerland')

#Index 1
print('the Dutch centroids with the Dutch routes')
retrieve_5_nn_info(1, 'Netherlands', 'Netherlands')
print('the Dutch centroids with the Swiss routes')
retrieve_5_nn_info(1, 'Netherlands', 'Switzerland')

#Index 2
print('the Dutch centroids with the Dutch routes')
retrieve_5_nn_info(2, 'Netherlands', 'Netherlands')
print('the Dutch centroids with the Swiss routes')
retrieve_5_nn_info(2, 'Netherlands', 'Switzerland')

#Index 3
print('the Dutch centroids with the Dutch routes')
retrieve_5_nn_info(3, 'Netherlands', 'Netherlands')
print('the Dutch centroids with the Swiss routes')
retrieve_5_nn_info(3, 'Netherlands', 'Switzerland')

#Index 4
print('the Dutch centroids with the Dutch routes')
retrieve_5_nn_info(4, 'Netherlands', 'Netherlands')
print('the Dutch centroids with the Swiss routes')
retrieve_5_nn_info(4, 'Netherlands', 'Switzerland')
    

#### Switzerland

In [ ]:
avg_abs_diff_tt_nn_swiss_centroid.rename(columns={'avg_abs_difference_tt_nn_Switzerland_tt_nn_Belgium': 'Belgium', 'avg_abs_difference_tt_nn_Switzerland_tt_nn_Netherlands': 'Netherlands'})

In [ ]:
''' Analysis for Belgian routes with Swiss centroids'''
#Index 2
print('the Swiss centroids with the Swiss routes')
retrieve_5_nn_info(2, 'Switzerland', 'Switzerland')
print('the Swiss centroids with the Belgian routes')
retrieve_5_nn_info(2, 'Switzerland', 'Belgium')

#Index 4
print('the Swiss centroids with the Swiss routes')
retrieve_5_nn_info(4, 'Switzerland', 'Switzerland')
print('the Swiss centroids with the Belgian routes')
retrieve_5_nn_info(4, 'Switzerland', 'Belgium')

In [ ]:
''' Analysis for Swiss routes with Swiss centroids'''
#Index 0
print('the Swiss centroids with the Swiss routes')
retrieve_5_nn_info(0, 'Switzerland', 'Switzerland')
print('the Swiss centroids with the Dutch routes')
retrieve_5_nn_info(0, 'Switzerland', 'Netherlands')

#Index 1
print('the Swiss centroids with the Swiss routes')
retrieve_5_nn_info(1, 'Switzerland', 'Switzerland')
print('the Swiss centroids with the Dutch routes')
retrieve_5_nn_info(1, 'Switzerland', 'Netherlands')

#Index 2
print('the Swiss centroids with the Swiss routes')
retrieve_5_nn_info(2, 'Switzerland', 'Switzerland')
print('the Swiss centroids with the Dutch routes')
retrieve_5_nn_info(2, 'Switzerland', 'Netherlands')

#Index 3
print('the Swiss centroids with the Swiss routes')
retrieve_5_nn_info(3, 'Switzerland', 'Switzerland')
print('the Swiss centroids with the Dutch routes')
retrieve_5_nn_info(3, 'Switzerland', 'Netherlands')

#Index 4
print('the Swiss centroids with the Swiss routes')
retrieve_5_nn_info(4, 'Switzerland', 'Switzerland')
print('the Swiss centroids with the Dutch routes')
retrieve_5_nn_info(4, 'Switzerland', 'Netherlands')

# DataFrames for Appendix

In [ ]:
route_id_nn_belgian_centroid
route_id_nn_dutch_centroid
route_id_nn_swiss_centroid

In [ ]:
get_centroid_2D_coord(cluster_centroids_2D_Belgium_from_3_countries)

In [ ]:
get_centroid_2D_coord(cluster_centroids_2D_Netherlands_from_3_countries)

In [ ]:
get_centroid_2D_coord(cluster_centroids_2D_Switzerland_from_3_countries)